In [10]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [11]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/cms/processed/metadata')
db_path=os.path.join(root_dir,'db/cms.db')

In [12]:
#define input tables and metadata
tbl_names_lst=['s1_ben_sum_2008','s1_ben_sum_2009','s1_ben_sum_2010','s1_carrier_1a','s1_carrier_1b','s1_inpatient','s1_outpatient','s1_prescrp']

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [13]:
# #setup parameters
# DFLT_PARAMS={
#     'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
#     'LOGIC_OPS':{'AND':0.9,'OR':0.1},
#     'NOT_STATE':{'0':0.8, '1':0.2},
#     'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
#     'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
#     'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
#     'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
#     'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
#     'JOIN_CNDTN':{'WHERE':0.5, 'AND':0.5} #Use WHERE or AND with JOIN CLAUSE
# }

#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'FILTER_TYPE':{'WHERE':0.5, 'AND':0.5}, #Use WHERE or AND with JOIN CLAUSE
    'JOIN_TYPE': {'JOIN':0.5, 'LEFT JOIN':0.5}
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

Metadata fot table s1_ben_sum_2008 validated.
Metadata fot table s1_ben_sum_2009 validated.
Metadata fot table s1_ben_sum_2010 validated.
Metadata fot table s1_carrier_1a validated.
Metadata fot table s1_carrier_1b validated.
Metadata fot table s1_inpatient validated.
Metadata fot table s1_outpatient validated.
Metadata fot table s1_prescrp validated.


# Generate Random  queries 

In [14]:
#Single Agg

# self.max_in_terms=2
self.no_groupby_vars=2
# self.no_where_vars=2
# self.no_join_tables=np.nan


rnd_query=self.make_single_agg_query(agg_fntn=True)
n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT s1_inpatient.ICD9_DGNS_CD_10, s1_inpatient.CLM_DRG_CD, COUNT(*), SUM(s1_inpatient.AT_PHYSN_NPI)  FROM s1_ben_sum_2009  LEFT JOIN s1_prescrp  ON s1_ben_sum_2009.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  JOIN s1_inpatient  ON s1_ben_sum_2009.DESYNPUF_ID=s1_inpatient.DESYNPUF_ID  LEFT JOIN s1_outpatient  ON s1_ben_sum_2009.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  GROUP BY s1_inpatient.ICD9_DGNS_CD_10, s1_inpatient.CLM_DRG_CD
no_of_rows=1136
no_of_cols=4
product=4.544


SELECT s1_inpatient.ICD9_DGNS_CD_10, s1_inpatient.CLM_DRG_CD, COUNT(*), SUM(s1_inpatient.AT_PHYSN_NPI)  FROM s1_ben_sum_2009  LEFT JOIN s1_prescrp  ON s1_ben_sum_2009.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  JOIN s1_inpatient  ON s1_ben_sum_2009.DESYNPUF_ID=s1_inpatient.DESYNPUF_ID  LEFT JOIN s1_outpatient  ON s1_ben_sum_2009.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  GROUP BY s1_inpatient.ICD9_DGNS_CD_10, s1_inpatient.CLM_DRG_CD


In [15]:
# Twin Aggregate

# self.max_in_terms=2
self.no_groupby_vars=2
# self.no_where_vars=2
# self.no_join_tables=np.nan

syn_tbl_name_lst=['s2_ben_sum_2008','s2_ben_sum_2009','s2_ben_sum_2010','s2_carrier_2a','s2_carrier_2b','s2_inpatient','s2_outpatient','s2_prescrp']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst,agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])


REAL:
no_of_rows=0
no_of_cols=4
product=0.0


SYN:
no_of_rows=0
no_of_cols=4
product=0.0


SELECT s1_inpatient.NCH_BENE_BLOOD_DDCTBL_LBLTY_AM, s1_outpatient.NCH_PRMRY_PYR_CLM_PD_AMT, COUNT(*), AVG(s1_inpatient.HCPCS_CD_16)  FROM s1_ben_sum_2008  JOIN s1_outpatient  ON s1_ben_sum_2008.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  LEFT JOIN s1_inpatient  ON s1_ben_sum_2008.DESYNPUF_ID=s1_inpatient.DESYNPUF_ID  JOIN s1_carrier_1a  ON s1_ben_sum_2008.DESYNPUF_ID=s1_carrier_1a.DESYNPUF_ID  JOIN s1_carrier_1b  ON s1_ben_sum_2008.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  GROUP BY s1_inpatient.NCH_BENE_BLOOD_DDCTBL_LBLTY_AM, s1_outpatient.NCH_PRMRY_PYR_CLM_PD_AMT


SELECT s2_inpatient.NCH_BENE_BLOOD_DDCTBL_LBLTY_AM, s2_outpatient.NCH_PRMRY_PYR_CLM_PD_AMT, COUNT(*), AVG(s2_inpatient.HCPCS_CD_16)  FROM s2_ben_sum_2008  JOIN s2_outpatient  ON s2_ben_sum_2008.DESYNPUF_ID=s2_outpatient.DESYNPUF_ID  LEFT JOIN s2_inpatient  ON s2_ben_sum_2008.DESYNPUF_ID=s2_inpatient.DESYNPUF_ID  JOIN s2_carrier_2a  ON s2_ben_

In [16]:
# Single Filter

# self.max_in_terms=2
# self.no_groupby_vars=2
self.no_where_vars=2
# self.no_join_tables=np.nan

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT *  FROM s1_ben_sum_2008  LEFT JOIN s1_carrier_1b  ON s1_ben_sum_2008.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  JOIN s1_prescrp  ON s1_ben_sum_2008.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  WHERE   s1_ben_sum_2008.PPPYMT_CAR NOT IN ('0.00', '0.00', '0.00') AND NOT  s1_carrier_1b.LINE_ALOWD_CHRG_AMT_13 NOT IN ('0.00', '0.00', '0.00') 
no_of_rows=0
no_of_cols=182
product=0.0


SELECT *  FROM s1_ben_sum_2008  LEFT JOIN s1_carrier_1b  ON s1_ben_sum_2008.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  JOIN s1_prescrp  ON s1_ben_sum_2008.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  WHERE   s1_ben_sum_2008.PPPYMT_CAR NOT IN ('0.00', '0.00', '0.00') AND NOT  s1_carrier_1b.LINE_ALOWD_CHRG_AMT_13 NOT IN ('0.00', '0.00', '0.00') 


In [17]:
# Twin Filter

self.no_where_vars=2

syn_tbl_name_lst=['s2_ben_sum_2008','s2_ben_sum_2009','s2_ben_sum_2010','s2_carrier_2a','s2_carrier_2b','s2_inpatient','s2_outpatient','s2_prescrp']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

SELECT *  FROM s1_ben_sum_2010  LEFT JOIN s1_carrier_1b  ON s1_ben_sum_2010.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  JOIN s1_prescrp  ON s1_ben_sum_2010.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  JOIN s1_outpatient  ON s1_ben_sum_2010.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  AND   s1_carrier_1b.ICD9_DGNS_CD_5 <> 'V4582' AND  s1_outpatient.HCPCS_CD_13 = '80164' 
REAL:
no_of_rows=0
no_of_cols=258
product=0.0


SYN:
no_of_rows=0
no_of_cols=258
product=0.0


SELECT *  FROM s1_ben_sum_2010  LEFT JOIN s1_carrier_1b  ON s1_ben_sum_2010.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  JOIN s1_prescrp  ON s1_ben_sum_2010.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  JOIN s1_outpatient  ON s1_ben_sum_2010.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  AND   s1_carrier_1b.ICD9_DGNS_CD_5 <> 'V4582' AND  s1_outpatient.HCPCS_CD_13 = '80164' 


SELECT *  FROM s2_ben_sum_2010  LEFT JOIN s2_carrier_2b  ON s2_ben_sum_2010.DESYNPUF_ID=s2_carrier_2b.DESYNPUF_ID  JOIN s2_prescrp  ON s2_ben_sum_2010.DESYNPUF_ID=s2_prescrp.DESYNPUF_ID  JOIN s2_ou

In [18]:
rnd_query['query_desc']

{'type': 'twin_fltr',
 'from_tbl_name_real': 's1_ben_sum_2010',
 'join_tbl_name_lst_real': ['s1_carrier_1b', 's1_prescrp', 's1_outpatient'],
 'sql_real': "SELECT *  FROM s1_ben_sum_2010  LEFT JOIN s1_carrier_1b  ON s1_ben_sum_2010.DESYNPUF_ID=s1_carrier_1b.DESYNPUF_ID  JOIN s1_prescrp  ON s1_ben_sum_2010.DESYNPUF_ID=s1_prescrp.DESYNPUF_ID  JOIN s1_outpatient  ON s1_ben_sum_2010.DESYNPUF_ID=s1_outpatient.DESYNPUF_ID  AND   s1_carrier_1b.ICD9_DGNS_CD_5 <> 'V4582' AND  s1_outpatient.HCPCS_CD_13 = '80164' ",
 'n_cols_real': 258,
 'n_rows_real': 0,
 'from_tbl_name_syn': 's2_ben_sum_2010',
 'join_tbl_name_lst_syn': ['s2_carrier_2b', 's2_prescrp', 's2_outpatient'],
 'sql_syn': "SELECT *  FROM s2_ben_sum_2010  LEFT JOIN s2_carrier_2b  ON s2_ben_sum_2010.DESYNPUF_ID=s2_carrier_2b.DESYNPUF_ID  JOIN s2_prescrp  ON s2_ben_sum_2010.DESYNPUF_ID=s2_prescrp.DESYNPUF_ID  JOIN s2_outpatient  ON s2_ben_sum_2010.DESYNPUF_ID=s2_outpatient.DESYNPUF_ID  AND   s2_carrier_2b.ICD9_DGNS_CD_5 <> 'V4582' AND  s2_o

In [ ]:
#Single Aggregate-Filter

# self.max_in_terms=2
self.no_groupby_vars=2
# self.no_where_vars=2
# self.no_join_tables=np.nan


rnd_query=self.make_single_aggfltr_query(agg_fntn=True)
n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

In [ ]:
# Twin Aggregate-Filter

# self.max_in_terms=2
self.no_groupby_vars=2
# self.no_where_vars=2
# self.no_join_tables=np.nan

syn_tbl_name_lst=['s2_ben_sum_2008','s2_ben_sum_2009','s2_ben_sum_2010','s2_carrier_2a','s2_carrier_2b','s2_inpatient','s2_outpatient','s2_prescrp']
rnd_query=self.make_twin_aggfltr_query(syn_tbl_name_lst,agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])